## Setup

To make the creation of strategies as simple as possible, WDSS has created a Python module which handles gameplay and provides a base strategy for you to work from. The game and base strategy are imported in the following code block.

In [2]:
import os
import sys

if 'google.colab' in str(get_ipython()):
    if not os.path.exists('game-theory-beyond-the-book'):
        !git clone https://github.com/warwickdatascience/game-theory-beyond-the-book.git
    else:
        !git -C game-theory-beyond-the-book pull
    sys.path.append('game-theory-beyond-the-book/src') 
else:
    sys.path.append('../src')
    
from gtbtb import Game, BaseStrategy

If you want to use any other packages/modules, you can import them here. For example, we will use the `random` module for a random strategy. Make sure you only include packages/modules that are part of the standard Python library.

In [3]:
import random

## Example Strategies

### Tit-for-Tat

A classic stategy for split-or-steal is tit-for-tat. This plays the opponents move back at them with the hope of teaching them to "play nicely". To implement such a strategy, we will have to access information about the game state. Specifically, we will need to know what turn we are currently on (so we can play split by default on the first turn) as well as the history of opponents moves. The first we access using

```python
self.game.get_history(self.op_id, "action")
```

This will give us a list of all moved previously played by our opponent. We could replace `op_id` with `my_id` to obtain our own history or replace `"action"` with `"intention"` or `"response"` to obtain a record of what the opponent _said_ they would do (more on this in a coming notebook). Those familiar with Python will know that we can obtain the last history entry using `[-1]`.

The finally strategy will look something like the following.

In [4]:
class TitForTat(BaseStrategy):
    
    def state_action(self):
        if self.game.get_turn() == 1:
            return 0  # start playing nice
        return self.game.get_history(self.op_id, "action")[-1]

This strategy is not willing to be pushed around by a nasty strategy (apart from the first round in which it gives the benefit of the doubt).

In [5]:
class Nasty(BaseStrategy):
    
    def state_action(self):
        return 1

In [6]:
Game(strategies=(TitForTat, Nasty), iterations=100).get_results()

{0: 0.0, 1: 1.0}

### Alternator

We can also access our own previous actions by replacing `self.op_id` with `self.my_id`. For example, we can make a strategy that alternates splitting and stealing.

In [7]:
class Alternator(BaseStrategy):
    
    def state_action(self):
        if self.game.get_turn() == 1:
            return 0  # start playing nice
        return self.game.get_history(self.my_id, "action")[-1]

### Nice-for-Now

Finitely-iterated games are particularly of interest because there is incentive to deviate from a fair strategy. After all, the closer we are to finishing the game, the less one's reputation matters. For example, we may want to create a strategy that plays nice until the final move whenever the game is finite (always playing nice in infinite games to be on the safe side).

In [8]:
class NiceForNow(BaseStrategy):
    
    def state_action(self):
        if not self.game.get_infinite() and \
            self.game.get_turn() == self.game.get_iterations():
            return 1
        return 0

In [9]:
class Nice(BaseStrategy):
    
    def state_action(self):
        return 0

In [10]:
Game(strategies=(Nice, NiceForNow), iterations=10).get_results()

{0: 45.0, 1: 55.0}

### Grudger

Strategies are allowed to maintain their own internal state, additional to that provided by the game. For example, we could write a strategy that plays nice until the opponent steals, at which point it steals forever. To initialise the strategy's internal state we have to define an `__init__(self, game, id_)` method. In this, we can create attributes to monitor the state using `self.attr_name = attr_value`. Finally, we close the `__init__` block using `super().__init__(game, id_)`. We won't get into what all of this means; just trust us that you need it!

In [11]:
class Grudger(BaseStrategy):
    
    def __init__(self, game, id_):
        self.grudging = False
        super().__init__(game, id_)
    
    def state_action(self):
        # Start nice
        if self.game.get_turn() == 1:
            return 0
        # Already grudging
        if self.grudging:
            return 1
        # Starting to grudge
        if self.game.get_history(self.op_id, "action")[-1] == 1:
            self.grudging = True
            return 1
        # No grudging yet
        return 0

In [12]:
class Random(BaseStrategy):
    
    def state_action(self):
        action = random.choice((0, 1))
        return action

In [13]:
Game(strategies=(Grudger, Random), iterations=100).get_results()

{0: 53.0, 1: 1.0}